In [1]:
# the following line is necessary to have interactive plots. In other case, all plots will be inline (and not interactive)
%matplotlib qt 
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import mne
from mne_bids import BIDSPath, read_raw_bids # you have to install mne_bids library with pip install mne-bids
from mne.channels import make_standard_montage

### Loading the data (in BIDS format)

The data are from the **"EEG Resting state EEG with closed eyes and open eyes in females from 60 to 80 years old database"** that can be found here [https://openneuro.org/datasets/ds005420/versions/1.0.0].

More on the data can be found here:
1. Appelhoff, S., Sanderson, M., Brooks, T., Vliet, M., Quentin, R., Holdgraf, C., Chaumon, M., Mikulan, E., Tavabi, K., Höchenberger, R., Welke, D., Brunner, C., Rockhill, A., Larson, E., Gramfort, A. and Jas, M. (2019). MNE-BIDS: Organizing electrophysiological data into the BIDS format and facilitating their analysis. Journal of Open Source Software 4: (1896). https://doi.org/10.21105/joss.01896

2. Pernet, C. R., Appelhoff, S., Gorgolewski, K. J., Flandin, G., Phillips, C., Delorme, A., Oostenveld, R. (2019). EEG-BIDS, an extension to the brain imaging data structure for electroencephalography. Scientific Data, 6, 103. https://doi.org/10.1038/s41597-019-0104-81.

In order to be able to load the EEG recordings of the subjects you need to define the correct path in the ```bids_root``` variable and the corresponding subject number in the ```subject``` variable.

The result of the following block of code is two data arrays:
1. ```raw_open``` with the raw EEG recordings while the subject has their eyes open
2. ```raw_closed``` with the raw EEG recordings while the subject has their eyes closed


In [2]:
# Set path to BIDS dataset
bids_root = Path().cwd().parent / "Resting_State_Database"  # Path to the root of the BIDS dataset
subject = "1"  # Replace with assigned subject number (without "sub-" prefix)

# Define BIDS paths for eyes open and eyes closed conditions
bids_path_open = BIDSPath(
    subject=subject,
    task="oa",  # 'oa' is the task code for eyes open
    datatype="eeg",
    root=bids_root
)

bids_path_closed = BIDSPath(
    subject=subject,
    task="oc",  # 'oc' is the task code for eyes closed
    datatype="eeg",
    root=bids_root
)

# Load the data using BIDS paths
raw_open = read_raw_bids(bids_path_open, verbose=True)
raw_closed = read_raw_bids(bids_path_closed, verbose=True)

# Then load the data into memory
raw_open.load_data()
raw_closed.load_data()


Extracting EDF parameters from c:\Users\Mateusz\PycharmProjects\signal_processing\lec_3\Resting_State_Database\sub-1\eeg\sub-1_task-oa_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading channel info from c:\Users\Mateusz\PycharmProjects\signal_processing\lec_3\Resting_State_Database\sub-1\eeg\sub-1_task-oa_channels.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Extracting EDF parameters from c:\Users\Mateusz\PycharmProjects\signal_processing\lec_3\Resting_State_Database\sub-1\eeg\sub-1_task-oc_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading channel info from c:\Users\Mateusz\PycharmProjects\signal_processing\lec_3\Resting_State_Database\sub-1\eeg\sub-1_task-oc_channels.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


C:\Users\Mateusz\AppData\Local\Temp\ipykernel_12244\3585756261.py:21: RuntimeWarning: Did not find any events.tsv associated with sub-1_task-oa.

The search_str was "c:\Users\Mateusz\PycharmProjects\signal_processing\lec_3\Resting_State_Database\sub-1\**\eeg\sub-1*events.tsv"
  raw_open = read_raw_bids(bids_path_open, verbose=True)
C:\Users\Mateusz\AppData\Local\Temp\ipykernel_12244\3585756261.py:21: RuntimeWarning: Unable to map the following column(s) to to MNE:
sex (subject sex): None
  raw_open = read_raw_bids(bids_path_open, verbose=True)
C:\Users\Mateusz\AppData\Local\Temp\ipykernel_12244\3585756261.py:22: RuntimeWarning: Did not find any events.tsv associated with sub-1_task-oc.

The search_str was "c:\Users\Mateusz\PycharmProjects\signal_processing\lec_3\Resting_State_Database\sub-1\**\eeg\sub-1*events.tsv"
  raw_closed = read_raw_bids(bids_path_closed, verbose=True)
C:\Users\Mateusz\AppData\Local\Temp\ipykernel_12244\3585756261.py:22: RuntimeWarning: Unable to map the followin

Reading 0 ... 103999  =      0.000 ...   207.998 secs...
Reading 0 ... 105499  =      0.000 ...   210.998 secs...


<RawEDF | sub-1_task-oc_eeg.edf, 20 x 105500 (211.0 s), ~16.1 MiB, data loaded>

In [14]:
# The BIDS reader automatically loads metadata like channel types, sampling frequency, etc.
print(f"Sampling rate: {raw_open.info['sfreq']} Hz")
print(f"Number of channels: {len(raw_open.ch_names)}")
print(f"Channel names: {raw_open.ch_names}")

Sampling rate: 500.0 Hz
Number of channels: 20
Channel names: ['EEG Fp1-A1A2', 'EEG Fp2-A1A2', 'EEG Fz-A1A2', 'EEG F3-A1A2', 'EEG F4-A1A2', 'EEG F7-A1A2', 'EEG F8-A1A2', 'EEG Cz-A1A2', 'EEG C3-A1A2', 'EEG C4-A1A2', 'EEG T3-A1A2', 'EEG T4-A1A2', 'EEG Pz-A1A2', 'EEG P3-A1A2', 'EEG P4-A1A2', 'EEG T5-A1A2', 'EEG T6-A1A2', 'EEG O1-A1A2', 'EEG O2-A1A2', 'EEG LOC-ROC']


Now we will rename the channel names into something more typical following the 10-20 international system. In addition we will also rename the EOG channel (this will not be used in our analysis, but feel free to use it if you want to further process the data) into something more standard.

In [3]:
# Create a mapping from your channel names to standard 10-20 names
ch_mapping = {
    'EEG Fp1-A1A2': 'Fp1',
    'EEG Fp2-A1A2': 'Fp2',
    'EEG Fz-A1A2': 'Fz',
    'EEG F3-A1A2': 'F3',
    'EEG F4-A1A2': 'F4',
    'EEG F7-A1A2': 'F7',
    'EEG F8-A1A2': 'F8',
    'EEG Cz-A1A2': 'Cz',
    'EEG C3-A1A2': 'C3',
    'EEG C4-A1A2': 'C4',
    'EEG T3-A1A2': 'T7',  # or 'T7' in newer naming
    'EEG T4-A1A2': 'T8',  # or 'T8' in newer naming
    'EEG Pz-A1A2': 'Pz',
    'EEG P3-A1A2': 'P3',
    'EEG P4-A1A2': 'P4',
    'EEG T5-A1A2': 'P7',  # or 'P7' in newer naming
    'EEG T6-A1A2': 'P8',  # or 'P8' in newer naming
    'EEG O1-A1A2': 'O1',
    'EEG O2-A1A2': 'O2',
}

# First, explicitly set the channel type for the EOG channel
raw_open.set_channel_types({'EEG LOC-ROC': 'eog'})
raw_closed.set_channel_types({'EEG LOC-ROC': 'eog'})

# Now rename just the EEG channels
raw_open.rename_channels(ch_mapping)
raw_closed.rename_channels(ch_mapping)

# Set the montage with on_missing='ignore' to ignore the EOG channel
montage = make_standard_montage('standard_1020')
raw_open.set_montage(montage, on_missing='ignore')
raw_closed.set_montage(montage, on_missing='ignore')

# Plot to verify channel locations
fig = raw_open.plot_sensors(show_names=True)


Now we can plot our two data structures, eyes open / eyes closed using interactive plots of MNE.

In [4]:
raw_closed.plot()
raw_open.plot()

Using qt as 2D backend.


<mne_qt_browser._pg_figure.MNEQtBrowser(0x1a1ae8e24f0) at 0x000001A31068FD40>

Channels marked as bad:
none


c:\Users\Mateusz\anaconda3\envs\biosignals\lib\site-packages\mne_qt_browser\_pg_figure.py:3061: RuntimeWarning: Failed to disconnect (None) from signal "triggered()".
  sig.disconnect()


Channels marked as bad:
none


### Signal Preprocessing

In this part we present the basic workflow of the signal pre-processing, that includes:
1. Bandpass filtering keeping frequencies in the range of 1-40Hz (since we don't really care for higher frequencies in our analysis).\
2. Notch filtering the mains frequency (50Hz) for Europe.

Note that since we are particularly interested in analyzing the brain activity inside the alpha (8-13Hz) band, we could have band-pass filtered our recordings using lower value for the upper-frequency of the filter (i.e. 30Hz)

In [ ]:
# Preprocessing
# 1. Apply bandpass filter (1-40 Hz)
raw_open_filtered = raw_open.copy().filter(l_freq=1, h_freq=40, 
                                         method='fir', phase='zero',
                                         fir_design='firwin')

raw_closed_filtered = raw_closed.copy().filter(l_freq=1, h_freq=40, 
                                             method='fir', phase='zero',
                                             fir_design='firwin')

# 2. Apply notch filter for line noise (50 Hz)
raw_open_filtered.notch_filter(freqs=50, picks='eeg')
raw_closed_filtered.notch_filter(freqs=50, picks='eeg')

# 3. Identify bad channels
# Option 1: Visual inspection
raw_open_filtered.plot(duration=30, n_channels=20)  # Look for bad channels
raw_closed_filtered.plot(duration=30, n_channels=20)

plt.show()

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 1651 samples (3.302 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Channels marked as bad:
none
Channels marked as bad:
none


### Spectral Analysis

In this part, we perform the spectral analysis of the signals. 

1. First we plot the Power Spectral Density (PSD) of the signals in both conditions (eyes open/eyes closed) using ```fmin=0Hz``` to ```fmax=48Hz``` for each channel. Finally we plot the **average** PSD using all channels for each condition.
2. We make a selection of EEG electrodes near the region of interest. Since we are interested for alpha activity which is located at the back of the brain (occipital), we don't want the signal to analyze and plot to be "contaminated" by brain activity not directly related to the eyes open/closed experiment. For this, we consider only the parietal and occipital electrodes and in particular the P3, P4, Pz, O1 and O2 and plot the PSD for each one of these.
3. Then, we plot the topography of the power as this is distributed on the electrode space (topographic plot) in different basic frequency bands (delta - theta - alpha - beta - gamma). This is very informative, as it can reveal the area that significant PSD is located (we expect this area to be at the back of the head for the eyes closed condition).
4. Finally we show each channel's PSD relative to its position on the head. This plot is interactive, as we can click on any PSD and have it zoomed in a separate window.

In [ ]:
# First we estimate and then plot the mean PSD across all electrodes for both the conditions 
spectrum_open = raw_open_filtered.compute_psd(fmin=0, fmax = 48)
spectrum_open.plot(average=True, picks="data", exclude="bads", amplitude=False)
plt.title("EEG. PSD. Eyes Open")

spectrum_closed = raw_closed_filtered.compute_psd(fmin=0, fmax = 48)
spectrum_closed.plot(average=True, picks="data", exclude="bads", amplitude=False)
plt.title("EEG. PSD. Eyes Closed")

# Next we define a region of interest on the electrode space and plot the power of each electrode
channel_sel = ['O1', 'O2', 'P3', 'P4', 'Pz']
spectrum_open.plot(picks=channel_sel, exclude="bads", amplitude=False)
plt.title("EEG. Open.")

spectrum_closed.plot(picks=channel_sel, exclude="bads", amplitude=False)
plt.title("EEG. Closed.")

# Topographic plots of the distribution of the power on the electrode space for both conditions.
spectrum_open.plot_topomap()
plt.title("EEG. Open.")

spectrum_closed.plot_topomap()
plt.title("EEG. Closed.")

# Finally we plot all channels' PSD plots in a way relative to their position on the head
spectrum_open.plot_topo()
plt.title("EEG. Open.")
spectrum_closed.plot_topo()
plt.title("EEG. Closed.")


plt.show()


Effective window size : 4.096 (s)
Plotting power spectral density (dB=True).
Effective window size : 4.096 (s)
Plotting power spectral density (dB=True).
Plotting power spectral density (dB=True).
Plotting power spectral density (dB=True).


1. In this part, we continue estimating the PSD of the signals for both conditions, using specific parameters in the calculation of the PSD, using the Welch method. Then, we plot the PSD of the O1, O2 channels for both conditions on a plot that has highlighted the alpha frequency band (8-13Hz). These plors don't differentiate from the previous that rely solely on MNE's functions and methods, but with a little extra cost programmatically, we can shape the plots in the way we want (to be more informative).

2. We calculate the mean power for each channel in both cases, but only **inside the alpha band** and we print the power ratio for each channel as

3. Finally we plot alpha power topography for eyes open, eyes closed as well as the power_ratio for each channel

In [19]:
# 1. Calculate PSD using Welch's method


# Define frequency range
fmin, fmax = 0.5, 45

# Calculate PSD for both conditions
psd_open, freqs = raw_open_filtered.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)

psd_closed, freqs = raw_closed_filtered.compute_psd(method='welch', fmin=fmin, fmax=fmax).get_data(return_freqs=True)


# 2. Plot PSD comparison
# Select occipital and parietal channels where alpha is typically most prominent
picks = ['O1', 'O2']
pick_idx = [raw_open_filtered.ch_names.index(ch) for ch in picks]

plt.figure(figsize=(12, 8))
# Plot individual channels
for idx, ch_idx in enumerate(pick_idx):
    plt.semilogy(freqs, psd_open[ch_idx], 
                label=f'{raw_open_filtered.ch_names[ch_idx]} (Open)', 
                alpha=0.5, linestyle='--')
    plt.semilogy(freqs, psd_closed[ch_idx], 
                label=f'{raw_closed_filtered.ch_names[ch_idx]} (Closed)', 
                alpha=0.5)

# Highlight alpha band
plt.axvspan(8, 13, color='yellow', alpha=0.2, label='Alpha Band (8-13 Hz)')

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power Spectral Density (µV²/Hz)')
plt.title('Power Spectrum: Eyes Open vs Eyes Closed')
plt.legend(loc='upper right')
plt.grid(True)
plt.xlim(fmin, 30)  # Limit x-axis to 30 Hz for better visualization
plt.show()

# 3. Calculate alpha band power
alpha_band = (8, 13)
alpha_idx = np.logical_and(freqs >= alpha_band[0], freqs <= alpha_band[1])

# Extract alpha power for all channels
alpha_power_open = np.mean(psd_open[:, alpha_idx], axis=1)
alpha_power_closed = np.mean(psd_closed[:, alpha_idx], axis=1)
alpha_ratio = alpha_power_closed / alpha_power_open

# Print alpha power for each channel
print("Alpha Power (8-13 Hz):")
for i, ch_name in enumerate(raw_open_filtered.ch_names[:-1]):
    print(f"{ch_name}:")
    print(f"  Eyes Open: {alpha_power_open[i]:.2f} µV²/Hz")
    print(f"  Eyes Closed: {alpha_power_closed[i]:.2f} µV²/Hz")
    print(f"  Ratio (Closed/Open): {alpha_ratio[i]:.2f}x")

# 4. Generate topographic maps
# Plot alpha power topography for eyes open
mne.viz.plot_topomap(alpha_power_open, raw_open_filtered.info, 
                    show=False, cmap='viridis')
plt.title('Alpha Power - Eyes Open')

# Plot alpha power topography for eyes closed
mne.viz.plot_topomap(alpha_power_closed, raw_closed_filtered.info, 
                    show=False, cmap='viridis')
plt.title('Alpha Power - Eyes Closed')

# Plot ratio of alpha power (closed/open)
mne.viz.plot_topomap(alpha_ratio, raw_open_filtered.info, 
                    show=False, cmap='viridis')
plt.title('Alpha Power Ratio (Closed/Open)')
plt.show()

Effective window size : 4.096 (s)


Effective window size : 4.096 (s)
Alpha Power (8-13 Hz):
Fp1:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 0.99x
Fp2:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 0.91x
Fz:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.14x
F3:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.23x
F4:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.13x
F7:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.14x
F8:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.17x
Cz:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.06x
C3:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.31x
C4:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.08x
T7:
  Eyes Open: 0.00 µV²/Hz
  Eyes Closed: 0.00 µV²/Hz
  Ratio (Closed/Open): 1.43x
T8:
  

### Time Frequency Analysis

Here we perform further analysis of our signals using Time Frequency analysis techniques. Since the signal that we examine cannot be considered to be stationary, we want to break it in smaller parts and perform the analysis therein.
For this purpose, we take only the electrode 'O1' (but you are more than welcome to analyze any other electrode you like) and we break it into smaller time windows with duration of 2sec (again feel free to play with the duration and see how the results change).
Next we perform time-frequency analysis using Wavelet analysis (a common method of multiresolution analysis) inside a range of frequencies 4-30Hz.

In [20]:
# Time-Frequency Analysis
from mne.time_frequency import tfr_morlet

# Select a posterior channel where alpha is prominent
ch_name = 'O1'
ch_idx = raw_open_filtered.ch_names.index(ch_name)

# Create epochs for analysis (divide continuous data into 2-second epochs)
# This helps visualize time-frequency content over the recording
epoch_duration = 2.0  # seconds
events_open = mne.make_fixed_length_events(raw_open_filtered, 
                                         duration=epoch_duration)
events_closed = mne.make_fixed_length_events(raw_closed_filtered,
                                           duration=epoch_duration)

epochs_open = mne.Epochs(raw_open_filtered, events_open, tmin=0, tmax=epoch_duration,
                       baseline=None, preload=True, reject=None)
epochs_closed = mne.Epochs(raw_closed_filtered, events_closed, tmin=0, tmax=epoch_duration,
                         baseline=None, preload=True, reject=None)

# Define frequencies of interest
freqs = np.arange(4, 30, 1)  # 4-30 Hz in 1 Hz steps
n_cycles = freqs / 2.  # Adjust cycles for better time-frequency resolution

# Compute time-frequency representation with Morlet wavelets

power_open = epochs_open.compute_tfr(
    method="morlet",
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=False,
    decim=3,
)
power_closed = epochs_closed.compute_tfr(
    method="morlet",
    freqs=freqs,
    n_cycles=n_cycles,
    average=True,
    return_itc=False,
    decim=3,
)



Not setting metadata
104 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 104 events and 1001 original time points ...
1 bad epochs dropped
Not setting metadata
105 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 105 events and 1001 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.6s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.7s


After calculating the power of the signals in different frequencies and the different 2sec time intervals, we proceed and plot them. What we would like *ideally* to see is the power to be significantly larger in the alpha band for the eyes closed condition compared to the eyes open condition.

In [22]:
# Plot time-frequency representations

power_open.plot([ch_idx], baseline=None, mode='mean', 
               title=f'Time-Frequency Analysis: {ch_name} - Eyes Open',
               show=False)

power_closed.plot([ch_idx], baseline=None, mode='mean', 
                 title=f'Time-Frequency Analysis: {ch_name} - Eyes Closed',
                 show=False)
plt.show()



No baseline correction applied
No baseline correction applied


Next we want to show a comparative plot of the mean power of the channel we analyze within the frequency band of interest (alpha band 8-13Hz). 

This plot is different from the previous, as it shows the **mean** power across all epochs (*remember the 2sec blocks we formed earlier?*) for the two different conditions (eyes open/closed). This results in very informative plots as *theoretically* we should see that the power of the occipital channel for the eyes closed condition should be **greater** than the power of the signal in the eyes open condition

In [23]:
# Plot alpha band time course (power over time)
alpha_freqs = np.logical_and(freqs >= 8, freqs <= 13)
alpha_idx = np.where(alpha_freqs)[0]

plt.figure(figsize=(12, 6))
plt.plot(power_open.times, np.mean(power_open.data[ch_idx, alpha_idx, :], axis=0),
         label='Eyes Open', color='blue')
plt.plot(power_closed.times, np.mean(power_closed.data[ch_idx, alpha_idx, :], axis=0),
         label='Eyes Closed', color='red')
plt.xlabel('Time (s)')
plt.ylabel('Alpha Power')
plt.title(f'Alpha Power Over Time: {ch_name}')
plt.legend()
plt.grid(True)
plt.show()